<a href="https://colab.research.google.com/github/khaledwaleedsamir/optimize-and-deploy-DL-models/blob/mobilenet/MobileNet_Trained2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow_datasets as tfds
# Import Dataset
(train_dataset, dev_dataset, test_dataset), info = tfds.load(
    'cats_vs_dogs',
    # Split the dataset 80% for training and 10% for dev and 10% for test
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
    with_info=True,
    as_supervised=True
)
# Verify dataset splits
print(f"Number of training samples: {len(train_dataset)}")
print(f"Number of validation samples: {len(dev_dataset)}")
print(f"Number of test samples: {len(test_dataset)}")

Number of training samples: 18610
Number of validation samples: 2326
Number of test samples: 2326


In [ ]:
import tensorflow as tf
# Normalize the image pixels to be from 0 to 1 (float32)
def normalize_img(image, label):
  return (tf.cast(image, tf.float32) / 255.0, label)
# Resize the image to 224x224
def resize(image, label):
  return (tf.image.resize(image, (224, 224)), label)

In [ ]:
# Preprocess the training set
SHUFFLE_VAL = len(train_dataset) // 100
BATCH_SIZE = 32
train_dataset = train_dataset.map(resize, num_parallel_calls=tf.data.AUTOTUNE)
train_dataset = train_dataset.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
train_dataset = train_dataset.shuffle(SHUFFLE_VAL)
train_dataset = train_dataset.batch(BATCH_SIZE)
train_dataset = train_dataset.prefetch(tf.data.AUTOTUNE)

In [ ]:
for x, y in train_dataset:
  print(x.shape, y.numpy())
  image_1 = x.numpy()
  break

(32, 224, 224, 3) [1 0 1 1 0 0 1 0 1 1 1 1 1 1 1 1 1 1 0 0 0 1 0 1 0 1 0 1 1 0 0 0]


In [ ]:
# Preprocess the dev set
dev_dataset = dev_dataset.map(resize, num_parallel_calls=tf.data.AUTOTUNE)
dev_dataset = dev_dataset.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
dev_dataset = dev_dataset.batch(BATCH_SIZE)
dev_dataset = dev_dataset.prefetch(tf.data.AUTOTUNE)

# Preprocess the test set
test_dataset = test_dataset.map(resize, num_parallel_calls=tf.data.AUTOTUNE)
test_dataset = test_dataset.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE)
test_dataset = test_dataset.prefetch(tf.data.AUTOTUNE)

In [ ]:
import tensorflow as tf
# Import all necessary layers
from tensorflow.keras import layers, Model, regularizers
from tensorflow.keras.layers import Input, DepthwiseConv2D
from tensorflow.keras.layers import Conv2D, BatchNormalization, ZeroPadding2D
from tensorflow.keras.layers import ReLU, AvgPool2D, Flatten, Dense, GlobalAveragePooling2D

In [ ]:
from tensorflow.keras.layers import Input, Conv2D, DepthwiseConv2D, BatchNormalization, ReLU, ZeroPadding2D
from tensorflow.keras.models import Model

def MobileNet():
    input_layer = Input(shape=(224, 224, 3), name='input_layer')

    # First convolutional block
    x = Conv2D(32, kernel_size=3, strides=(2, 2), padding='same', name='conv1', use_bias = False)(input_layer)
    x = BatchNormalization(name='conv1_bn')(x)
    x = ReLU(name='conv1_relu')(x)

    # Depthwise Separable Convolutional blocks
    def depthwise_separable_conv_block(inputs, pointwise_conv_filters, block_id, strides=(1, 1)):
        x = DepthwiseConv2D(kernel_size=3, strides=strides, padding='same', name=f'conv_dw_{block_id}', use_bias = False)(inputs)
        x = BatchNormalization(name=f'conv_dw_{block_id}_bn')(x)
        x = ReLU(name=f'conv_dw_{block_id}_relu')(x)

        x = Conv2D(pointwise_conv_filters, kernel_size=1, padding='same', name=f'conv_pw_{block_id}', use_bias = False)(x)
        x = BatchNormalization(name=f'conv_pw_{block_id}_bn')(x)
        x = ReLU(name=f'conv_pw_{block_id}_relu')(x)

        return x

    # Add the depthwise separable convolutional blocks
    x = depthwise_separable_conv_block(x, 64, block_id=1)
    x = ZeroPadding2D(padding=((1, 0), (1, 0)), name='conv_pad_2')(x)
    x = depthwise_separable_conv_block(x, 128, block_id=2, strides=(2, 2))
    x = depthwise_separable_conv_block(x, 128, block_id=3)
    x = ZeroPadding2D(padding=((1, 1), (1, 1)), name='conv_pad_4')(x)
    x = depthwise_separable_conv_block(x, 256, block_id=4, strides=(2, 2))
    x = depthwise_separable_conv_block(x, 256, block_id=5)
    x = ZeroPadding2D(padding=((1, 1), (1, 1)), name='conv_pad_6')(x)
    x = depthwise_separable_conv_block(x, 512, block_id=6, strides=(2, 2))

    for i in range(7, 12):
        x = depthwise_separable_conv_block(x, 512, block_id=i)

    x = ZeroPadding2D(padding=((1, 1), (1, 1)), name='conv_pad_12')(x)
    x = depthwise_separable_conv_block(x, 1024, block_id=12, strides=(2, 2))
    x = depthwise_separable_conv_block(x, 1024, block_id=13)
    # Add custom classification layers for binary classification
    x = GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.5)(x)  # Dropout layer to prevent overfitting
    x = Dense(128, activation='relu')(x)
    output = Dense(1, activation='sigmoid')(x)  # Binary classification (cats vs dogs)
    model = Model(inputs=input_layer, outputs=output)
    return model

model = MobileNet()
model.summary()


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1 (Conv2D)                       │ (None, 112, 112, 32)        │             864 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1_bn (BatchNormalization)        │ (None, 112, 112, 32)        │             128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1_relu (ReLU)                    │ (None, 112, 112, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_dw_1 (DepthwiseConv2D)          │ (None, 112, 112, 32)        │             288 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_dw_1_bn (BatchNormalization)    │ (None, 112, 112, 32)        │             128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_dw_1_relu (ReLU)                │ (None, 112, 112, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_pw_1 (Conv2D)                   │ (None, 112, 112, 64)        │           2,048 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_pw_1_bn (BatchNormalization)    │ (None, 112, 112, 64)        │             256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_pw_1_relu (ReLU)                │ (None, 112, 112, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_pad_2 (ZeroPadding2D)           │ (None, 113, 113, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_dw_2 (DepthwiseConv2D)          │ (None, 57, 57, 64)          │             576 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_dw_2_bn (BatchNormalization)    │ (None, 57, 57, 64)          │             256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_dw_2_relu (ReLU)                │ (None, 57, 57, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_pw_2 (Conv2D)                   │ (None, 57, 57, 128)         │           8,192 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_pw_2_bn (BatchNormalization)    │ (None, 57, 57, 128)         │             512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_pw_2_relu (ReLU)                │ (None, 57, 57, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_dw_3 (DepthwiseConv2D)          │ (None, 57, 57, 128)         │           1,152 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_dw_3_bn (BatchNormalization)    │ (None, 57, 57, 128)         │             512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_dw_3_relu (ReLU)                │ (None, 57, 57, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_pw_3 (Conv2D)                   │ (None, 57, 57, 128)         │          16,384 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 3,360,193 (12.82 MB)

 Trainable params: 3,338,305 (12.73 MB)

 Non-trainable params: 21,888 (85.50 KB)

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.losses import BinaryCrossentropy
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,  # Number of epochs to wait before stopping
    restore_best_weights=True,  # Restore weights from the best epoch
    verbose=1
)
# Define a learning rate scheduler
lr_scheduler = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,  # Reduce learning rate by half
    patience=3,  # Wait for 3 epochs before reducing learning rate
    min_lr=1e-6,  # Lower bound of the learning rate
    verbose=1
)

In [ ]:
# Train the Model
model.fit(train_dataset, epochs=50, validation_data=dev_dataset, callbacks=[early_stopping, lr_scheduler])

Epoch 1/50
582/582 ━━━━━━━━━━━━━━━━━━━━ 108s 140ms/step - accuracy: 0.5678 - loss: 0.6998 - val_accuracy: 0.4815 - val_loss: 0.6971 - learning_rate: 1.0000e-04
Epoch 2/50
582/582 ━━━━━━━━━━━━━━━━━━━━ 67s 114ms/step - accuracy: 0.6577 - loss: 0.6163 - val_accuracy: 0.6638 - val_loss: 0.6104 - learning_rate: 1.0000e-04
Epoch 3/50
582/582 ━━━━━━━━━━━━━━━━━━━━ 67s 114ms/step - accuracy: 0.6986 - loss: 0.5786 - val_accuracy: 0.6647 - val_loss: 0.6000 - learning_rate: 1.0000e-04
Epoch 4/50
582/582 ━━━━━━━━━━━━━━━━━━━━ 83s 115ms/step - accuracy: 0.7258 - loss: 0.5433 - val_accuracy: 0.7124 - val_loss: 0.5766 - learning_rate: 1.0000e-04
Epoch 5/50
582/582 ━━━━━━━━━━━━━━━━━━━━ 67s 114ms/step - accuracy: 0.7523 - loss: 0.5047 - val_accuracy: 0.6836 - val_loss: 0.6171 - learning_rate: 1.0000e-04
Epoch 6/50
582/582 ━━━━━━━━━━━━━━━━━━━━ 83s 117ms/step - accuracy: 0.7776 - loss: 0.4663 - val_accuracy: 0.6758 - val_loss: 0.7113 - learning_rate: 1.0000e-04
Epoch 7/50
582/582 ━━━━━━━━━━━━━━━━━━━━ 0s 10